In [10]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from ta import add_all_ta_features
from ta.trend import ema_indicator
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import datetime

from strategy import *

In [11]:
# Grid search
xg_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'max_depth': [3, 4, 5]
        }

In [12]:
def run_cerebro(dataname,train_start,random_grid,strategy):
    df = pd.read_csv(dataname+'.csv',index_col=0)
    
    df.index = pd.to_datetime(df.index)
    X_train, y_train = df.loc[train_start:'2018-01-01',:].iloc[:,:-2], df.loc[train_start:'2018-01-01']['state']
    X_test, y_test = df.loc['2018-01-01':'2022-01-01',:].iloc[:,:-2], df.loc['2018-01-01':'2022-01-01']['state']

    xg = XGBClassifier()
    xg_random = RandomizedSearchCV(estimator = xg, param_distributions = random_grid, n_iter = 100, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
    xg_random.fit(X_train, y_train)
    params = xg_random.best_params_

    model = XGBClassifier(**params).fit(X_train, y_train)

    # model = XGBClassifier().fit(X_train, y_train)
    y_pred = model.predict(X_test)

    score = model.score(X_test,y_test)
    f1 = f1_score(y_pred,y_test,average='weighted')

    df.loc['2018-01-01':'2022-01-01',['predict']] = y_pred.reshape(-1,1)


    # Backtesting
    # df['Date']=pd.to_datetime(df.index)
    df = df.reset_index()
    state_column = df.shape[1]-1
        # Instantiate Cerebro engine
    cerebro = bt.Cerebro()

    data = PandasData(
        dataname = df,
        fromdate=datetime.datetime(2018, 1, 2),
        todate=datetime.datetime(2021, 12, 30),
        datetime = 0,
        open = 1,
        high = 2,
        low = 3,
        close = 4,
        state = state_column,
    )

    # Add data to Cerebro
    cerebro.adddata(data)

    # Add strategy to Cerebro
    cerebro.addstrategy(strategy)

    # Default position size
    cerebro.addsizer(bt.sizers.SizerFix, stake=1)

    # Add analytics to Cerebro
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='SharpeRatio')
    cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='AnnualReturn')
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='DrawDown')
    cerebro.addanalyzer(btanalyzers.TimeDrawDown, _name='TimeDrawDown')
    cerebro.addanalyzer(btanalyzers.PositionsValue, _name='PositionsValue')
    cerebro.addanalyzer(btanalyzers.LogReturnsRolling, _name='LogReturnsRolling')
    cerebro.addanalyzer(btanalyzers.PeriodStats, _name='PeriodStats')
    cerebro.addanalyzer(btanalyzers.Returns, _name='Returns')
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='TradeAnalyzer')
    cerebro.addanalyzer(btanalyzers.Transactions, _name='Transactions')

    # Run Cerebro Engine
    start_portfolio_value = cerebro.broker.getvalue()

    # cerebro.run()
    thestrats = cerebro.run()
    thestrat = thestrats[0]

    end_portfolio_value = cerebro.broker.getvalue()
    pnl = end_portfolio_value - start_portfolio_value

    print(f'Starting Portfolio Value: {start_portfolio_value:2f}')
    print(f'Final Portfolio Value: {end_portfolio_value:2f}')
    print(f'PnL: {pnl:.2f}\n')


    sharpe = thestrat.analyzers.SharpeRatio.get_analysis()['sharperatio']
    # print(thestrat.analyzers.AnnualReturn.get_analysis())
    # print(thestrat.analyzers.DrawDown.get_analysis())
    # # print(thestrat.analyzers.TimeDrawDown.get_analysis())
    # print(thestrat.analyzers.PositionsValue.get_analysis())
    # print(thestrat.analyzers.LogReturnsRolling.get_analysis())
    # print(thestrat.analyzers.PeriodStats.get_analysis())
    # print(thestrat.analyzers.Returns.get_analysis())
    # print(thestrat.analyzers.TradeAnalyzer.get_analysis())
    # print(thestrat.analyzers.Transactions.get_analysis())
    return pnl, sharpe, score, f1, params

In [13]:
# datanames = ['y1_2003','y2_2003','y3_2003','y4_2003','y5_2003','gmm_1_labeled_2003','hmm_1_labeled_2003']
datanames = ['y1_2003','y2_2003','y3_2003','y4_2003','y5_2003','gmm_1_labeled_2003','hmm_1_labeled_2003']
train_start_list = ['2003-01-01','2010-01-04','2015-01-02']

pnl_result = []
pnl = 0
for dataname in datanames:
    for train_start in train_start_list:
        pnl, sharpe, score, f1, params = run_cerebro(dataname,train_start,xg_grid,MarketStatus)
        pnl_dic = {'dataname':dataname,
                    'train_start':train_start,
                    'pnl':pnl,
                    'sharpe':sharpe,
                    'score':score,
                    'f1':f1,
                    'params':params
                    }
        pnl_result.append(pnl_dic)
pnl_result

C:\Users\lemon\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 45 is smaller than n_iter=100. Running 45 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 45 candidates, totalling 225 fits


In [ ]:
pnl_result_df = pd.DataFrame(pnl_result)
pnl_result_df.sort_values(by='pnl',ascending=False)
pnl_result_df.to_excel('0 xg_result.xlsx')